In [0]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
!wc /content/gdrive/My\ Drive/Colab\ Notebooks/Data/week2/golden.csv
# !head -n 100000 /content/gdrive/My\ Drive/Colab\ Notebooks/Data/week2/golden.csv > /content/gdrive/My\ Drive/Colab\ Notebooks/Data/week7/100000golden.csv
# !du -sh /content/gdrive/My\ Drive/Colab\ Notebooks/Data/week7/100000golden.csv

   507918   1015835 105699993 /content/gdrive/My Drive/Colab Notebooks/Data/week2/golden.csv


In [0]:
df = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/Data/week2/golden.csv')

In [0]:
N0 = 507917
N = 507917 - 1000

H_i = 100
n_attributes = 8
N_image = N - (H_i-1) # 507890
X_conv2d = True
y = df['Target']
y_target = np.roll(y, -(H_i -1))[:N_image]

# [y_roll] N, to N, 2

df.describe()

,Timestamp,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price,Weighted_Price_MA60,WP_MA1_mins,WP_MA5_mins,WP_MA15_mins,WP_MA30_mins,WP_MA60_mins,Delta,Target
count,5.079170e+05,5.079170e+05,5.079170e+05,5.079170e+05,5.079170e+05,5.079170e+05,5.079170e+05,5.079170e+05,5.079170e+05,507917.000000,507917.000000,507917.000000,507917.000000,507917.000000,507917.000000,507917.000000
mean,1.514748e+09,8.797389e+05,8.805404e+05,8.789189e+05,8.797387e+05,1.296136e+01,1.170516e+07,8.797113e+05,8.796891e+05,0.996834,0.989128,0.978218,0.967537,0.952362,-0.183510,0.481222
std,8.879069e+06,4.387864e+05,4.393738e+05,4.381860e+05,4.387830e+05,1.649170e+01,1.700317e+07,4.387680e+05,4.387485e+05,0.006127,0.016220,0.027411,0.038334,0.051576,1787.577142,0.499648
min,1.499375e+09,2.062910e+05,2.064800e+05,2.060000e+05,2.063460e+05,5.600000e-07,2.274938e-01,2.063754e+05,2.088214e+05,0.897054,0.809287,0.726925,0.693798,0.628358,-80875.000000,0.000000
25%,1.506993e+09,5.039300e+05,5.041170e+05,5.036990e+05,5.039800e+05,3.160630e+00,2.192749e+06,5.039470e+05,5.038718e+05,0.996452,0.985792,0.969498,0.953987,0.928369,-401.000000,0.000000
50%,1.514820e+09,8.383290e+05,8.387760e+05,8.379960e+05,8.383540e+05,7.641194e+00,5.961625e+06,8.383088e+05,8.383658e+05,0.999328,0.995353,0.988112,0.980528,0.969623,0.000000,0.000000
75%,1.522439e+09,1.047999e+06,1.048886e+06,1.047000e+06,1.047999e+06,1.633734e+01,1.425951e+07,1.047936e+06,1.048183e+06,1.000000,0.999305,0.996872,0.994514,0.990275,399.000000,1.000000
max,1.530058e+09,2.312098e+06,2.312100e+06,2.300290e+06,2.312098e+06,7.574348e+02,6.936638e+08,2.311319e+06,2.273014e+06,1.000000,1.000000,1.000000,1.000000,1.000000,129608.000000,1.000000


In [0]:
attributes_normalize_mean = ['Timestamp', 'Open', 'High', 'Low', 'Close', 'Weighted_Price']
attributes_normalize_log = ['Volume_(BTC)', 'Volume_(Currency)']

n_attributes_total = attributes_normalize_mean + attributes_normalize_log

X_normalized = pd.DataFrame()

# standardize

for att in attributes_normalize_mean:

    float_array = df[att].values.astype(float)
    x_mean = float_array.mean()

    std = np.std(float_array)

    float_array = (float_array - x_mean) / std
    X_normalized[att] = float_array


for att in attributes_normalize_log:

    float_array = df[att].values.astype(float)
    
    float_array = np.log(float_array)

    x_mean = float_array.mean()

    std = np.std(float_array)

    float_array = (float_array - x_mean) / std
    X_normalized[att] = float_array


x_values = X_normalized.values #returns a numpy array


X_image = np.empty((N_image, H_i, n_attributes), dtype=float)

for idx in range(N_image):
    X_image[idx] = x_values[idx:idx+H_i]
    

print(X_image.shape)

(506818, 100, 8)


In [0]:
# if X_conv2d is True and len(X_image.shape) is not 4:
#   X_image = np.expand_dims(X_image, axis=3)
# X_image.shape

In [0]:
# momentum_bn = 1-1024/430000
def create_model():
  
  model = tf.keras.models.Sequential([
    keras.layers.Conv1D(filters=128, kernel_size=5, padding='same',
                        data_format="channels_last", activation=None, 
                        batch_input_shape=(None, 100, 8),use_bias=True),
    keras.layers.BatchNormalization(axis = -1),
      
#     keras.layers.BatchNormalization(momentum=momentum_bn, axis= -1),

    keras.layers.Activation('elu'),
      
    keras.layers.Conv1D(filters=64, kernel_size=3, padding='same',
                        data_format="channels_last", activation=None, 
                        batch_input_shape=(None, 100, 8),use_bias=True),
    keras.layers.BatchNormalization(axis = -1),
#     keras.layers.BatchNormalization(momentum=momentum_bn, axis= -1),

    keras.layers.Activation('elu'),

      
    keras.layers.Flatten(),
    keras.layers.BatchNormalization(axis = -1),
#     keras.layers.BatchNormalization(momentum=momentum_bn, axis= -1),
  

    keras.layers.Dense(64, activation='elu', use_bias=True),
    
    keras.layers.Dropout(0.4),
    keras.layers.Dense(2, activation=tf.nn.softmax)
  ])
  
  model.compile(
    #optimizer=tf.train.AdamOptimizer(learning_rate=0.0005, beta1=0.995, beta2=0.9995, epsilon=1e-8),

    optimizer=tf.train.AdamOptimizer(learning_rate=0.0002),
    loss=tf.keras.losses.sparse_categorical_crossentropy,
    metrics=['accuracy'])
  
  return model


model = create_model()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_2 (Conv1D)            (None, 100, 128)          5248      
_________________________________________________________________
batch_normalization_3 (Batch (None, 100, 128)          512       
_________________________________________________________________
activation_2 (Activation)    (None, 100, 128)          0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 100, 64)           24640     
_________________________________________________________________
batch_normalization_4 (Batch (None, 100, 64)           256       
_________________________________________________________________
activation_3 (Activation)    (None, 100, 64)           0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 6400)              0         
__________

In [0]:
# test GPU should return '/device:GPU:0'

tf.test.gpu_device_name()
# !mkdir /content/gdrive/My\ Drive/checkpoint/training_week10/2conv_batchnormalize_elu

'/device:GPU:0'

Load pretrain


In [0]:
import pathlib
import os
checkpoint_path = "/content/gdrive/My Drive/checkpoint/training_week10/2conv_batchnormalize_elu_low_eta/cp.ckpt"

checkpoint_dir = os.path.dirname(checkpoint_path)
# Sort the checkpoints by modification time.
checkpoints = pathlib.Path(checkpoint_dir).glob("*.index")
model = create_model()

checkpoints = sorted(checkpoints, key=lambda cp:cp.stat().st_mtime)
checkpoints = [cp.with_suffix('') for cp in checkpoints]
latest = str(checkpoints[-1])
latest


# load pre-trained weights
model.load_weights(latest)

IndexError: ignored

In [0]:
#N = 150000
# create pre-trained

from time import time
import os

N = 507000
N0 = 430000
N_val = 1024
N_time_update = 1024
N_loop = (N - N0) // N_time_update - 10  ### -10 just make sure


cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, 
                                                 save_weights_only=True,
                                                 verbose=1)

metrics_file_save = '/content/gdrive/My Drive/checkpoint/training_week10/2conv_batchnormalize_elu_low_eta/metrics_eval.txt'
metrics = np.array([])
metrics = np.loadtxt(metrics_file_save, delimiter=',')

for each_time in range(N_loop):
    t0 = time()

    X_train = X_image[:N0 + each_time * N_time_update]
    X_val = X_image[N0 + each_time * N_time_update + H_i:\
                    N0 + each_time * N_time_update + N_val + H_i]

    y_train = y_target[:N0 + each_time * N_val]
    y_val = y_target[N0 + each_time * N_time_update + H_i:\
                    N0 + each_time * N_time_update + N_val + H_i]


    model.fit(X_train,y_train,
              batch_size=1024,epochs=5,
              validation_data=(X_val, y_val),
              callbacks=[cp_callback]
             )
    metric_new = model.evaluate(X_val, y_val)
    metric_new = np.expand_dims(np.asarray(metric_new), axis=0)
    # reshape if empty
    metrics = np.concatenate((metrics.reshape(-1,2), metric_new)) 
  
    t1 = time()
    print("Time: {} mins {} secs".format((t1 - t0) // 60, (t1 - t0) % 60 // 1))

Train on 430000 samples, validate on 1024 samples
Epoch 1/5
  2048/430000 [..............................] - ETA: 30s - loss: 0.5700 - acc: 0.7026

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: UserWarning: loadtxt: Empty input file: "/content/gdrive/My Drive/checkpoint/training_week10/2conv_batchnormalize_elu_low_eta/metrics_eval.txt"


429056/430000 [============================>.] - ETA: 0s - loss: 0.5708 - acc: 0.6879
Epoch 00001: saving model to /content/gdrive/My Drive/checkpoint/training_week10/2conv_batchnormalize_elu_low_eta/cp.ckpt
430000/430000 [==============================] - 27s 63us/step - loss: 0.5707 - acc: 0.6879 - val_loss: 0.6454 - val_acc: 0.5693
Epoch 2/5
429056/430000 [============================>.] - ETA: 0s - loss: 0.5441 - acc: 0.7078
Epoch 00002: saving model to /content/gdrive/My Drive/checkpoint/training_week10/2conv_batchnormalize_elu_low_eta/cp.ckpt
430000/430000 [==============================] - 27s 63us/step - loss: 0.5441 - acc: 0.7078 - val_loss: 0.6560 - val_acc: 0.5576
Epoch 3/5
429056/430000 [============================>.] - ETA: 0s - loss: 0.5320 - acc: 0.7186
Epoch 00003: saving model to /content/gdrive/My Drive/checkpoint/training_week10/2conv_batchnormalize_elu_low_eta/cp.ckpt
430000/430000 [==============================] - 27s 63us/step - loss: 0.5318 - acc: 0.7187 - val_

In [0]:
np.mean(metrics, axis=0)
np.savetxt(metrics_file_save, metrics, delimiter=',')

In [0]:
metrics.shape

(0,)